# Predictions - Try #1


- Author: Elmo Chavez
- Date: 19-Jul-2023

**Description**

> [...]


## Read the Data


### Import Libraries


In [1]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, StratifiedKFold, StratifiedShuffleSplit
from sklearn.feature_selection import SelectKBest, mutual_info_classif, SelectPercentile, f_classif

import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import lightgbm as lgb

### Reading Datasets


In [2]:
current_directory = os.getcwd()  # Get the current working directory
parent_directory = os.path.dirname(current_directory)  # Get the parent directory

In [3]:
# Filenames
#filename_participants = 'participants_selected.csv'
filename_epochs_fp1 = '1_psd_features_fp1.csv'
filename_epochs_all = '1_psd_features_allch.csv'
filename_epochs_bnds_fp1 = '1_psd_bands_features_fp1.csv'
filename_epochs_bdns_all = '1_psd_bands_features_allch.csv'

# Participants info
# df_participants = pd.read_csv(parent_directory+'/Training Datasets/'+filename_participants)

# Epochs based feature extraction
df_epochs_fp1 = pd.read_csv(parent_directory+'/Training Datasets/'+filename_epochs_fp1)
df_epochs_all = pd.read_csv(parent_directory+'/Training Datasets/'+filename_epochs_all)

# Epochs and Frequency Bands feature extraction
df_epochs_bands_fp1 = pd.read_csv(parent_directory+'/Training Datasets/'+filename_epochs_bnds_fp1)
df_epochs_bands_all = pd.read_csv(parent_directory+'/Training Datasets/'+filename_epochs_bdns_all)

# Shape of the data
#print('Participants info:',df_participants.shape)
print('Epochs base Feature Extraction - FP1:',df_epochs_fp1.shape)
print('Epochs base Feature Extraction - All Channels:',df_epochs_all.shape)
print('Epochs base with Freq. Bands Feature Extraction - FP1:',df_epochs_bands_fp1.shape)
print('Epochs base with Freq. Bands Feature Extraction - All Channels:',df_epochs_bands_all.shape)

Epochs base Feature Extraction - FP1: (714, 11)
Epochs base Feature Extraction - All Channels: (714, 11)
Epochs base with Freq. Bands Feature Extraction - FP1: (714, 31)
Epochs base with Freq. Bands Feature Extraction - All Channels: (714, 31)


## Grouping Data


In [4]:
features = []

#### Epoch based FP1


In [5]:
df_epochs_fp1.head()

,subject_id,age,gender,epoch_id,total_channels,total_power,avg_power,relative_power,peak_freq,spectral_entropy,group
0,sub-002,78,1,0,19.0,8.138737e-07,3.255495e-08,0.480204,1.953125,0.000013,0
1,sub-002,78,1,1,19.0,6.457802e-07,2.583121e-08,0.373726,1.953125,0.000010,0
2,sub-002,78,1,2,19.0,7.862074e-07,3.144830e-08,0.397717,1.953125,0.000013,0
3,sub-002,78,1,3,19.0,8.496011e-07,3.398405e-08,0.458611,1.953125,0.000013,0
4,sub-002,78,1,4,19.0,7.503180e-07,3.001272e-08,0.402417,1.953125,0.000012,0


In [6]:
df_epochs_fp1_grp = df_epochs_fp1.groupby('subject_id').mean().reset_index()
print(df_epochs_fp1_grp.shape)
df_epochs_fp1_grp.head()

(51, 11)


,subject_id,age,gender,epoch_id,total_channels,total_power,avg_power,relative_power,peak_freq,spectral_entropy,group
0,sub-002,78.0,1.0,6.5,19.0,8.747903e-07,3.499161e-08,0.440120,1.953125,0.000014,0.0
1,sub-004,67.0,1.0,6.5,19.0,1.032087e-06,4.128349e-08,0.459969,1.953125,0.000016,0.0
2,sub-005,70.0,0.0,6.5,19.0,1.147346e-06,4.589383e-08,0.461923,1.953125,0.000017,0.0
3,sub-006,61.0,1.0,6.5,19.0,1.349809e-06,5.399235e-08,0.489254,1.953125,0.000020,0.0
4,sub-007,79.0,1.0,6.5,19.0,9.680973e-07,3.872389e-08,0.437120,1.953125,0.000015,0.0


In [7]:
cols_to_drop_fp1 = ['subject_id','group','epoch_id','total_channels','peak_freq']

features.append(df_epochs_fp1_grp.drop(columns=cols_to_drop_fp1, axis=1).columns.tolist())

X_epochs_fp1 = df_epochs_fp1_grp.drop(columns=cols_to_drop_fp1, axis=1).values
y_epochs_fp1 = df_epochs_fp1_grp['group'].values

print(X_epochs_fp1.shape)
df_epochs_fp1_grp.drop(columns=cols_to_drop_fp1, axis=1).head()

(51, 6)


,age,gender,total_power,avg_power,relative_power,spectral_entropy
0,78.0,1.0,8.747903e-07,3.499161e-08,0.440120,0.000014
1,67.0,1.0,1.032087e-06,4.128349e-08,0.459969,0.000016
2,70.0,0.0,1.147346e-06,4.589383e-08,0.461923,0.000017
3,61.0,1.0,1.349809e-06,5.399235e-08,0.489254,0.000020
4,79.0,1.0,9.680973e-07,3.872389e-08,0.437120,0.000015


#### Epochs based All Channels


In [8]:
df_epochs_all.head()

,subject_id,age,gender,epoch_id,total_channels,total_power,avg_power,relative_power,peak_freq,spectral_entropy,group
0,sub-002,78,1,0,19.0,8.346040e-07,3.338416e-08,0.478456,1.953125,0.000013,0
1,sub-002,78,1,1,19.0,6.761889e-07,2.704755e-08,0.374997,1.953125,0.000011,0
2,sub-002,78,1,2,19.0,8.086165e-07,3.234466e-08,0.394570,1.953125,0.000013,0
3,sub-002,78,1,3,19.0,8.440792e-07,3.376317e-08,0.447835,1.953125,0.000013,0
4,sub-002,78,1,4,19.0,7.673421e-07,3.069369e-08,0.416419,1.953125,0.000012,0


In [9]:
df_epochs_all_grp = df_epochs_all.groupby('subject_id').mean().reset_index()

features.append(df_epochs_all_grp.drop(columns=cols_to_drop_fp1, axis=1).columns.tolist())

X_epochs_all = df_epochs_all_grp.drop(columns=cols_to_drop_fp1, axis=1).values
y_epochs_all = df_epochs_all_grp['group'].values

print(X_epochs_all.shape)
df_epochs_all_grp.drop(columns=cols_to_drop_fp1, axis=1).head()

(51, 6)


,age,gender,total_power,avg_power,relative_power,spectral_entropy
0,78.0,1.0,8.914837e-07,3.565935e-08,0.438730,0.000014
1,67.0,1.0,1.007653e-06,4.030614e-08,0.450322,0.000015
2,70.0,0.0,1.022872e-06,4.091487e-08,0.451693,0.000016
3,61.0,1.0,1.233868e-06,4.935472e-08,0.481064,0.000019
4,79.0,1.0,9.319299e-07,3.727719e-08,0.445109,0.000014


#### Epochs based with Frequency Bands FP1


In [10]:
df_epochs_bands_fp1.head()

,epoch_id,total_channels,delta_total_power,delta_avg_power,delta_relative_power,delta_peak_freq,delta_spectral_entropy,theta_total_power,theta_avg_power,theta_relative_power,...,beta_spectral_entropy,gamma_total_power,gamma_avg_power,gamma_relative_power,gamma_peak_freq,gamma_spectral_entropy,subject_id,age,gender,group
0,0,19.0,5.356440e-07,2.678220e-07,0.316042,1.953125,0.000008,1.703684e-08,8.518421e-09,0.010052,...,0.000001,1.292570e-07,1.615713e-08,0.076265,44.921875,0.000002,sub-002,78,1,0
1,1,19.0,3.903346e-07,1.951673e-07,0.225894,1.953125,0.000006,2.239278e-08,1.119639e-08,0.012959,...,0.000002,1.108229e-07,1.385286e-08,0.064135,35.156250,0.000002,sub-002,78,1,0
2,2,19.0,5.090068e-07,2.545034e-07,0.257490,1.953125,0.000008,3.471327e-08,1.735663e-08,0.017560,...,0.000002,8.779028e-08,1.097378e-08,0.044410,37.109375,0.000002,sub-002,78,1,0
3,3,19.0,5.687169e-07,2.843584e-07,0.306991,1.953125,0.000008,2.759734e-08,1.379867e-08,0.014897,...,0.000001,1.364043e-07,1.705053e-08,0.073631,39.062500,0.000002,sub-002,78,1,0
4,4,19.0,5.352662e-07,2.676331e-07,0.287079,1.953125,0.000008,2.820804e-08,1.410402e-08,0.015129,...,0.000001,9.871103e-08,1.233888e-08,0.052942,42.968750,0.000002,sub-002,78,1,0


In [11]:
df_epochs_bands_fp1_grp = df_epochs_bands_fp1.groupby('subject_id').mean().reset_index()
variances = df_epochs_bands_fp1_grp.var()
variances[variances==0]

/var/folders/42/zkw31xg11x1f6dpl9zz6hmz00000gn/T/ipykernel_46749/4002112219.py:2: FutureWarning: The default value of numeric_only in DataFrame.var is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  variances = df_epochs_bands_fp1_grp.var()


epoch_id           0.0
total_channels     0.0
delta_peak_freq    0.0
dtype: float64

In [12]:
cols_to_drop_all = ['subject_id','group','epoch_id','total_channels','delta_peak_freq']
X_epochs_bands_fp1 = df_epochs_bands_fp1_grp.drop(columns=cols_to_drop_all, axis=1).values
y_epochs_bands_fp1 = df_epochs_bands_fp1_grp['group'].values

features.append(df_epochs_bands_fp1_grp.drop(columns=cols_to_drop_all, axis=1).columns.tolist())
print(X_epochs_bands_fp1.shape)
df_epochs_bands_fp1_grp.drop(columns=cols_to_drop_all, axis=1).head()

(51, 26)


,delta_total_power,delta_avg_power,delta_relative_power,delta_spectral_entropy,theta_total_power,theta_avg_power,theta_relative_power,theta_peak_freq,theta_spectral_entropy,alpha_total_power,...,beta_relative_power,beta_peak_freq,beta_spectral_entropy,gamma_total_power,gamma_avg_power,gamma_relative_power,gamma_peak_freq,gamma_spectral_entropy,age,gender
0,6.162917e-07,3.081458e-07,0.308468,0.000009,2.990883e-08,1.495442e-08,0.015320,6.835938,5.332261e-07,2.314431e-08,...,0.049007,19.670759,0.000002,9.302605e-08,1.162826e-08,0.047920,39.481027,0.000002,78.0,1.0
1,7.330833e-07,3.665417e-07,0.326442,0.000010,3.841179e-08,1.920590e-08,0.017403,6.556920,6.764621e-07,2.168472e-08,...,0.050945,23.297991,0.000002,9.888761e-08,1.236095e-08,0.043597,37.946429,0.000002,67.0,1.0
2,8.562397e-07,4.281199e-07,0.343816,0.000012,4.064498e-08,2.032249e-08,0.016384,6.975446,7.121021e-07,2.642452e-08,...,0.043073,22.321429,0.000002,9.064191e-08,1.133024e-08,0.037134,36.690848,0.000002,70.0,0.0
3,9.774772e-07,4.887386e-07,0.353442,0.000014,6.790989e-08,3.395495e-08,0.024048,6.277902,1.150462e-06,3.904878e-08,...,0.048213,22.042411,0.000002,1.100085e-07,1.375106e-08,0.040758,38.783482,0.000002,61.0,1.0
4,6.885473e-07,3.442737e-07,0.310674,0.000010,3.065200e-08,1.532600e-08,0.014079,6.835938,5.444660e-07,2.510195e-08,...,0.045433,23.158482,0.000002,1.002958e-07,1.253698e-08,0.045057,37.527902,0.000002,79.0,1.0


#### Epochs based with Frequency Bands All Channels


In [13]:
df_epochs_bands_all.head()

,epoch_id,total_channels,delta_total_power,delta_avg_power,delta_relative_power,delta_peak_freq,delta_spectral_entropy,theta_total_power,theta_avg_power,theta_relative_power,...,beta_spectral_entropy,gamma_total_power,gamma_avg_power,gamma_relative_power,gamma_peak_freq,gamma_spectral_entropy,subject_id,age,gender,group
0,0,19.0,5.607184e-07,2.803592e-07,0.321419,1.953125,0.000008,1.780230e-08,8.901150e-09,0.010167,...,0.000001,1.402790e-07,1.753488e-08,0.080389,41.118421,0.000002,sub-002,78,1,0
1,1,19.0,4.036695e-07,2.018348e-07,0.224124,1.953125,0.000006,2.129087e-08,1.064544e-08,0.011726,...,0.000002,1.166759e-07,1.458449e-08,0.064537,35.361842,0.000002,sub-002,78,1,0
2,2,19.0,5.249767e-07,2.624883e-07,0.256317,1.953125,0.000008,3.140269e-08,1.570135e-08,0.015305,...,0.000002,1.045114e-07,1.306392e-08,0.050853,37.109375,0.000002,sub-002,78,1,0
3,3,19.0,5.627172e-07,2.813586e-07,0.298903,1.953125,0.000008,2.816719e-08,1.408359e-08,0.014952,...,0.000002,1.184824e-07,1.481030e-08,0.062715,37.828947,0.000002,sub-002,78,1,0
4,4,19.0,5.368642e-07,2.684321e-07,0.291363,1.953125,0.000008,3.484860e-08,1.742430e-08,0.018918,...,0.000001,1.033295e-07,1.291619e-08,0.056063,39.268092,0.000002,sub-002,78,1,0


In [14]:
df_epochs_bands_all_grp = df_epochs_bands_all.groupby('subject_id').mean().reset_index()
variances = df_epochs_bands_all_grp.var()
variances[variances==0]

/var/folders/42/zkw31xg11x1f6dpl9zz6hmz00000gn/T/ipykernel_46749/1963283297.py:2: FutureWarning: The default value of numeric_only in DataFrame.var is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  variances = df_epochs_bands_all_grp.var()


epoch_id           0.0
total_channels     0.0
delta_peak_freq    0.0
dtype: float64

In [15]:
cols_to_drop_all = ['subject_id','group','epoch_id','total_channels','delta_peak_freq']
X_epochs_bands_all = df_epochs_bands_all_grp.drop(columns=cols_to_drop_all, axis=1).values
y_epochs_bands_all = df_epochs_bands_all_grp['group'].values

features.append(df_epochs_bands_all_grp.drop(columns=cols_to_drop_all, axis=1).columns.tolist())
print(X_epochs_bands_all.shape)
df_epochs_bands_all_grp.drop(columns=cols_to_drop_all, axis=1).head()

(51, 26)


,delta_total_power,delta_avg_power,delta_relative_power,delta_spectral_entropy,theta_total_power,theta_avg_power,theta_relative_power,theta_peak_freq,theta_spectral_entropy,alpha_total_power,...,beta_relative_power,beta_peak_freq,beta_spectral_entropy,gamma_total_power,gamma_avg_power,gamma_relative_power,gamma_peak_freq,gamma_spectral_entropy,age,gender
0,6.237628e-07,3.118814e-07,0.306003,0.000009,3.379866e-08,1.689933e-08,0.016632,6.784539,5.969269e-07,2.569446e-08,...,0.049147,19.567963,0.000002,9.253067e-08,1.156633e-08,0.046496,38.967047,0.000002,78.0,1.0
1,7.182049e-07,3.591024e-07,0.320871,0.000010,4.059222e-08,2.029611e-08,0.018286,6.336642,7.125114e-07,2.439143e-08,...,0.050013,22.189262,0.000002,9.048753e-08,1.131094e-08,0.039987,37.527902,0.000002,67.0,1.0
2,7.335661e-07,3.667830e-07,0.324358,0.000011,3.466311e-08,1.733155e-08,0.015147,6.718456,6.110561e-07,2.755472e-08,...,0.043824,20.985080,0.000002,9.959176e-08,1.244897e-08,0.044421,36.264979,0.000002,70.0,0.0
3,8.816662e-07,4.408331e-07,0.342492,0.000013,5.372953e-08,2.686477e-08,0.020618,6.277902,9.240921e-07,4.007805e-08,...,0.049755,21.131931,0.000002,1.065104e-07,1.331380e-08,0.042442,38.445724,0.000002,61.0,1.0
4,6.580038e-07,3.290019e-07,0.314150,0.000009,2.979564e-08,1.489782e-08,0.014413,7.012159,5.309690e-07,2.462742e-08,...,0.047956,22.380169,0.000002,9.772223e-08,1.221528e-08,0.046418,37.616013,0.000002,79.0,1.0


In [16]:
for f in features:
    print(f)

['age', 'gender', 'total_power', 'avg_power', 'relative_power', 'spectral_entropy']
['age', 'gender', 'total_power', 'avg_power', 'relative_power', 'spectral_entropy']
['delta_total_power', 'delta_avg_power', 'delta_relative_power', 'delta_spectral_entropy', 'theta_total_power', 'theta_avg_power', 'theta_relative_power', 'theta_peak_freq', 'theta_spectral_entropy', 'alpha_total_power', 'alpha_avg_power', 'alpha_relative_power', 'alpha_peak_freq', 'alpha_spectral_entropy', 'beta_total_power', 'beta_avg_power', 'beta_relative_power', 'beta_peak_freq', 'beta_spectral_entropy', 'gamma_total_power', 'gamma_avg_power', 'gamma_relative_power', 'gamma_peak_freq', 'gamma_spectral_entropy', 'age', 'gender']
['delta_total_power', 'delta_avg_power', 'delta_relative_power', 'delta_spectral_entropy', 'theta_total_power', 'theta_avg_power', 'theta_relative_power', 'theta_peak_freq', 'theta_spectral_entropy', 'alpha_total_power', 'alpha_avg_power', 'alpha_relative_power', 'alpha_peak_freq', 'alpha_spe

## Predictions


In [17]:
filename = '1_results.csv'
df_results = pd.read_csv(current_directory+'/'+filename)

def append_result(result):      
    index = len(df_results)
    df_results.loc[index] = result
    
def remove_items(df, approach, frequencies, channels):
    mask = (df['Feature Extraction'] != approach) | (df['Frequencies'] != frequencies) | (df['Channels'] != channels)
    filtered_df = df[mask]
    return filtered_df

def save_results(df, path, filename):
    df.to_csv(path+'/'+filename, index=False)  
    print('Results saved, records:',len(df))
        
print('Results: ', len(df_results))

def predictor(X, y, approach, frequencies, channels, features_names, selector_k, selector_perc):
    # Define the classifiers
    svm_classifier = SVC()
    random_forest_classifier = RandomForestClassifier()
    adaboost_classifier = AdaBoostClassifier()
    xgboost_classifier = xgb.XGBClassifier()
    lightgbm_classifier = lgb.LGBMClassifier()

    classifiers = [svm_classifier, random_forest_classifier, adaboost_classifier, 
                   xgboost_classifier, lightgbm_classifier]
    classifier_names = ['SVM', 'Random Forest', 'AdaBoost', 
                        'XGBoost', 'LightGBM']

    # Create a cross-validator with 5 folds
    n_splits = 5
    test_size = 0.2

    cv_method = [KFold(n_splits=n_splits, shuffle=True, random_state=42),
                StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42),
                StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=42)]
    cv_method_names = ['KFold','StratifiedKFold','StratifiedShuffleSplit']

    # Create the Feature Selector method
    feat_sel_method =   [SelectKBest(score_func=f_classif, k=selector_k), SelectKBest(score_func=mutual_info_classif, k=selector_k),
                        SelectPercentile(score_func=f_classif, percentile=selector_perc), SelectPercentile(score_func=mutual_info_classif, percentile=selector_perc)]
    feat_sel_method_names = ['SelectKBest - f_classif','SelectKBest - mutual_info_classif',
                            'SelectPercentile - f_classif', 'SelectPercentile - mutual_info_classif']

    for clf, clf_name in zip(classifiers, classifier_names):
        print('Running',clf_name)
        for cv, cv_name in zip(cv_method, cv_method_names):
            #print('Running',clf_name,cv_name)
            for selector, sel_name in zip(feat_sel_method, feat_sel_method_names):
                #print('Running',clf_name,cv_name,sel_name)
                scores = []
                for train_index, test_index in cv.split(X, y):
                    # Split the data into training and testing sets based on the current fold indices
                    X_train, X_test = X[train_index], X[test_index]
                    y_train, y_test = y[train_index], y[test_index]
                    # Feature Selection
                    X_train_selected = selector.fit_transform(X_train, y_train)
                    selected_index = selector.get_support()
                    X_test_selected = selector.transform(X_test)
                    # Train the classifier on the training data
                    clf.fit(X_train_selected, y_train)
                    
                    # Make predictions on the test data
                    y_pred = clf.predict(X_test_selected)
                    # Calculate accuracy for this fold and append to the scores list
                    accuracy = np.mean(y_pred == y_test)
                    scores.append(accuracy)
        
                #save results
                result = [approach, frequencies, channels, features_names, cv_name, sel_name, 
                          [features_names[i] for i, mask in enumerate(selected_index) if mask], 
                          clf_name, np.mean(scores)]
                #print(result)
                append_result(result)

df_results.head()

Results:  120


,Feature Extraction,Frequencies,Channels,Metrics,Cross Validate Method,Feature Selection,Metrics_Selected,ML Model,Accuracy
0,Time-Frequency Domain,1-50 Hz,FP1,"['age', 'gender', 'total_power', 'avg_power', ...",KFold,SelectKBest - f_classif,"['age', 'gender', 'relative_power', 'spectral_...",SVM,0.645455
1,Time-Frequency Domain,1-50 Hz,FP1,"['age', 'gender', 'total_power', 'avg_power', ...",KFold,SelectKBest - mutual_info_classif,"['total_power', 'avg_power', 'relative_power',...",SVM,0.645455
2,Time-Frequency Domain,1-50 Hz,FP1,"['age', 'gender', 'total_power', 'avg_power', ...",KFold,SelectPercentile - f_classif,"['age', 'gender']",SVM,0.645455
3,Time-Frequency Domain,1-50 Hz,FP1,"['age', 'gender', 'total_power', 'avg_power', ...",KFold,SelectPercentile - mutual_info_classif,"['relative_power', 'spectral_entropy']",SVM,0.645455
4,Time-Frequency Domain,1-50 Hz,FP1,"['age', 'gender', 'total_power', 'avg_power', ...",StratifiedKFold,SelectKBest - f_classif,"['age', 'gender', 'total_power', 'avg_power']",SVM,0.647273


### Epochs Based - FP1 Channel


In [19]:
# Variables for Results
approach = 'Time-Frequency Domain'
frequencies = '1-50 Hz'
channels = 'FP1'

print('Number of features:', len(features[0]))

# Clean df_results
df_results = remove_items(df_results, approach, frequencies, channels)
#Prediction
predictor(X_epochs_fp1, y_epochs_fp1,approach,frequencies,channels,features[0], 4, 30)
# Save results
save_results(df_results,current_directory,filename)
# Show new results
df_results[(df_results['Feature Extraction'] == approach) &  (df_results['Frequencies'] == frequencies) &  (df_results['Channels'] == channels)].sort_values('Accuracy', ascending=False)


Number of features: 6
Running SVM
Running Random Forest
Running AdaBoost
Running XGBoost
Running LightGBM
Results saved, records: 60


,Feature Extraction,Frequencies,Channels,Metrics,Cross Validate Method,Feature Selection,Metrics_Selected,ML Model,Accuracy
4,Time-Frequency Domain,1-50 Hz,FP1,"[age, gender, total_power, avg_power, relative...",StratifiedKFold,SelectKBest - f_classif,"[age, gender, total_power, avg_power]",SVM,0.647273
5,Time-Frequency Domain,1-50 Hz,FP1,"[age, gender, total_power, avg_power, relative...",StratifiedKFold,SelectKBest - mutual_info_classif,"[gender, total_power, avg_power, spectral_entr...",SVM,0.647273
6,Time-Frequency Domain,1-50 Hz,FP1,"[age, gender, total_power, avg_power, relative...",StratifiedKFold,SelectPercentile - f_classif,"[age, gender]",SVM,0.647273
7,Time-Frequency Domain,1-50 Hz,FP1,"[age, gender, total_power, avg_power, relative...",StratifiedKFold,SelectPercentile - mutual_info_classif,"[age, spectral_entropy]",SVM,0.647273
55,Time-Frequency Domain,1-50 Hz,FP1,"[age, gender, total_power, avg_power, relative...",StratifiedKFold,SelectPercentile - mutual_info_classif,[spectral_entropy],LightGBM,0.647273
54,Time-Frequency Domain,1-50 Hz,FP1,"[age, gender, total_power, avg_power, relative...",StratifiedKFold,SelectPercentile - f_classif,"[age, gender]",LightGBM,0.647273
53,Time-Frequency Domain,1-50 Hz,FP1,"[age, gender, total_power, avg_power, relative...",StratifiedKFold,SelectKBest - mutual_info_classif,"[age, total_power, avg_power, spectral_entropy]",LightGBM,0.647273
52,Time-Frequency Domain,1-50 Hz,FP1,"[age, gender, total_power, avg_power, relative...",StratifiedKFold,SelectKBest - f_classif,"[age, gender, total_power, avg_power]",LightGBM,0.647273
0,Time-Frequency Domain,1-50 Hz,FP1,"[age, gender, total_power, avg_power, relative...",KFold,SelectKBest - f_classif,"[age, gender, relative_power, spectral_entropy]",SVM,0.645455
1,Time-Frequency Domain,1-50 Hz,FP1,"[age, gender, total_power, avg_power, relative...",KFold,SelectKBest - mutual_info_classif,"[total_power, avg_power, relative_power, spect...",SVM,0.645455


### Epochs Based - All Channels


In [20]:
# Variables for Results
approach = 'Time-Frequency Domain'
frequencies = '1-50 Hz'
channels = 'All'

print('Number of features:', len(features[1]))

# Clean df_results
df_results = remove_items(df_results, approach, frequencies, channels)
#Prediction
predictor(X_epochs_all, y_epochs_all,approach,frequencies,channels,features[1], 5, 35)
# Save results
save_results(df_results,current_directory,filename)
# Show new results
df_results[(df_results['Feature Extraction'] == approach) &  (df_results['Frequencies'] == frequencies) &  (df_results['Channels'] == channels)].sort_values('Accuracy', ascending=False)

Number of features: 6
Running SVM
Running Random Forest
Running AdaBoost
Running XGBoost
Running LightGBM
Results saved, records: 120


,Feature Extraction,Frequencies,Channels,Metrics,Cross Validate Method,Feature Selection,Metrics_Selected,ML Model,Accuracy
77,Time-Frequency Domain,1-50 Hz,All,"[age, gender, total_power, avg_power, relative...",StratifiedKFold,SelectKBest - mutual_info_classif,"[gender, total_power, avg_power, relative_powe...",Random Forest,0.647273
64,Time-Frequency Domain,1-50 Hz,All,"[age, gender, total_power, avg_power, relative...",StratifiedKFold,SelectKBest - f_classif,"[age, gender, total_power, avg_power, spectral...",SVM,0.647273
115,Time-Frequency Domain,1-50 Hz,All,"[age, gender, total_power, avg_power, relative...",StratifiedKFold,SelectPercentile - mutual_info_classif,"[gender, spectral_entropy]",LightGBM,0.647273
114,Time-Frequency Domain,1-50 Hz,All,"[age, gender, total_power, avg_power, relative...",StratifiedKFold,SelectPercentile - f_classif,"[age, gender]",LightGBM,0.647273
60,Time-Frequency Domain,1-50 Hz,All,"[age, gender, total_power, avg_power, relative...",KFold,SelectKBest - f_classif,"[age, gender, total_power, avg_power, spectral...",SVM,0.645455
62,Time-Frequency Domain,1-50 Hz,All,"[age, gender, total_power, avg_power, relative...",KFold,SelectPercentile - f_classif,"[total_power, avg_power]",SVM,0.645455
63,Time-Frequency Domain,1-50 Hz,All,"[age, gender, total_power, avg_power, relative...",KFold,SelectPercentile - mutual_info_classif,"[relative_power, spectral_entropy]",SVM,0.645455
111,Time-Frequency Domain,1-50 Hz,All,"[age, gender, total_power, avg_power, relative...",KFold,SelectPercentile - mutual_info_classif,"[relative_power, spectral_entropy]",LightGBM,0.645455
110,Time-Frequency Domain,1-50 Hz,All,"[age, gender, total_power, avg_power, relative...",KFold,SelectPercentile - f_classif,"[total_power, avg_power]",LightGBM,0.645455
61,Time-Frequency Domain,1-50 Hz,All,"[age, gender, total_power, avg_power, relative...",KFold,SelectKBest - mutual_info_classif,"[gender, total_power, avg_power, relative_powe...",SVM,0.645455


### Epochs Based with Frequency Bands - FP1 Channel


In [18]:
# Variables for Results
approach = 'Time-Frequency Domain'
frequencies = 'Delta, Theta, Alpha, Beta, Gamma'
channels = 'FP1'

print('Number of features:', len(features[2]))

# Clean df_results
df_results = remove_items(df_results, approach, frequencies, channels)
#Prediction
predictor(X_epochs_bands_fp1, y_epochs_bands_fp1,approach,frequencies,channels,features[2], 8, 25)
# Save results
save_results(df_results,current_directory,filename)
# Show new results
df_results[(df_results['Feature Extraction'] == approach) &  (df_results['Frequencies'] == frequencies) &  (df_results['Channels'] == channels)].sort_values('Accuracy', ascending=False)

Number of features: 26
Running SVM
Running Random Forest
Running AdaBoost
Running XGBoost
Running LightGBM
Results saved, records: 180


,Feature Extraction,Frequencies,Channels,Metrics,Cross Validate Method,Feature Selection,Metrics_Selected,ML Model,Accuracy
142,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",FP1,"[delta_total_power, delta_avg_power, delta_rel...",StratifiedShuffleSplit,SelectPercentile - f_classif,"[theta_peak_freq, alpha_peak_freq, beta_peak_f...",Random Forest,0.672727
140,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",FP1,"[delta_total_power, delta_avg_power, delta_rel...",StratifiedShuffleSplit,SelectKBest - f_classif,"[delta_spectral_entropy, theta_peak_freq, alph...",Random Forest,0.654545
124,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",FP1,"[delta_total_power, delta_avg_power, delta_rel...",StratifiedKFold,SelectKBest - f_classif,"[theta_total_power, theta_avg_power, theta_rel...",SVM,0.647273
125,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",FP1,"[delta_total_power, delta_avg_power, delta_rel...",StratifiedKFold,SelectKBest - mutual_info_classif,"[delta_total_power, delta_avg_power, theta_rel...",SVM,0.647273
126,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",FP1,"[delta_total_power, delta_avg_power, delta_rel...",StratifiedKFold,SelectPercentile - f_classif,"[theta_total_power, theta_avg_power, theta_rel...",SVM,0.647273
127,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",FP1,"[delta_total_power, delta_avg_power, delta_rel...",StratifiedKFold,SelectPercentile - mutual_info_classif,"[theta_relative_power, beta_total_power, beta_...",SVM,0.647273
174,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",FP1,"[delta_total_power, delta_avg_power, delta_rel...",StratifiedKFold,SelectPercentile - f_classif,"[theta_total_power, theta_avg_power, theta_rel...",LightGBM,0.647273
120,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",FP1,"[delta_total_power, delta_avg_power, delta_rel...",KFold,SelectKBest - f_classif,"[theta_total_power, theta_avg_power, theta_pea...",SVM,0.645455
122,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",FP1,"[delta_total_power, delta_avg_power, delta_rel...",KFold,SelectPercentile - f_classif,"[theta_peak_freq, theta_spectral_entropy, alph...",SVM,0.645455
123,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",FP1,"[delta_total_power, delta_avg_power, delta_rel...",KFold,SelectPercentile - mutual_info_classif,"[theta_relative_power, beta_total_power, beta_...",SVM,0.645455


### Epochs Based with Frequency Bands - All Channel


In [19]:
# Variables for Results
approach = 'Time-Frequency Domain'
frequencies = 'Delta, Theta, Alpha, Beta, Gamma'
channels = 'All'

print('Number of features:', len(features[3]))

# Clean df_results
df_results = remove_items(df_results, approach, frequencies, channels)
#Prediction
predictor(X_epochs_bands_all, y_epochs_bands_all,approach,frequencies,channels,features[3], 8, 25)
# Save results
save_results(df_results,current_directory,filename)
# Show new results
df_results[(df_results['Feature Extraction'] == approach) &  (df_results['Frequencies'] == frequencies) &  (df_results['Channels'] == channels)].sort_values('Accuracy', ascending=False)

Number of features: 26
Running SVM
Running Random Forest
Running AdaBoost
Running XGBoost
Running LightGBM
Results saved, records: 240


,Feature Extraction,Frequencies,Channels,Metrics,Cross Validate Method,Feature Selection,Metrics_Selected,ML Model,Accuracy
228,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",All,"[delta_total_power, delta_avg_power, delta_rel...",KFold,SelectKBest - f_classif,"[delta_total_power, delta_avg_power, alpha_pea...",LightGBM,0.685455
231,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",All,"[delta_total_power, delta_avg_power, delta_rel...",KFold,SelectPercentile - mutual_info_classif,"[alpha_peak_freq, beta_total_power, beta_avg_p...",LightGBM,0.665455
184,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",All,"[delta_total_power, delta_avg_power, delta_rel...",StratifiedKFold,SelectKBest - f_classif,"[theta_relative_power, beta_peak_freq, gamma_t...",SVM,0.647273
187,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",All,"[delta_total_power, delta_avg_power, delta_rel...",StratifiedKFold,SelectPercentile - mutual_info_classif,"[delta_spectral_entropy, alpha_relative_power,...",SVM,0.647273
180,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",All,"[delta_total_power, delta_avg_power, delta_rel...",KFold,SelectKBest - f_classif,"[delta_total_power, delta_avg_power, alpha_pea...",SVM,0.645455
182,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",All,"[delta_total_power, delta_avg_power, delta_rel...",KFold,SelectPercentile - f_classif,"[alpha_peak_freq, gamma_total_power, gamma_avg...",SVM,0.645455
183,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",All,"[delta_total_power, delta_avg_power, delta_rel...",KFold,SelectPercentile - mutual_info_classif,"[alpha_peak_freq, beta_total_power, beta_avg_p...",SVM,0.645455
229,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",All,"[delta_total_power, delta_avg_power, delta_rel...",KFold,SelectKBest - mutual_info_classif,"[alpha_peak_freq, beta_total_power, beta_avg_p...",LightGBM,0.645455
181,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",All,"[delta_total_power, delta_avg_power, delta_rel...",KFold,SelectKBest - mutual_info_classif,"[alpha_relative_power, alpha_peak_freq, beta_t...",SVM,0.645455
230,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",All,"[delta_total_power, delta_avg_power, delta_rel...",KFold,SelectPercentile - f_classif,"[alpha_peak_freq, gamma_total_power, gamma_avg...",LightGBM,0.645455


### Results


In [20]:
df_results.sort_values('Accuracy', ascending=False).head(20)

,Feature Extraction,Frequencies,Channels,Metrics,Cross Validate Method,Feature Selection,Metrics_Selected,ML Model,Accuracy
228,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",All,"[delta_total_power, delta_avg_power, delta_rel...",KFold,SelectKBest - f_classif,"[delta_total_power, delta_avg_power, alpha_pea...",LightGBM,0.685455
142,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",FP1,"[delta_total_power, delta_avg_power, delta_rel...",StratifiedShuffleSplit,SelectPercentile - f_classif,"[theta_peak_freq, alpha_peak_freq, beta_peak_f...",Random Forest,0.672727
231,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",All,"[delta_total_power, delta_avg_power, delta_rel...",KFold,SelectPercentile - mutual_info_classif,"[alpha_peak_freq, beta_total_power, beta_avg_p...",LightGBM,0.665455
140,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",FP1,"[delta_total_power, delta_avg_power, delta_rel...",StratifiedShuffleSplit,SelectKBest - f_classif,"[delta_spectral_entropy, theta_peak_freq, alph...",Random Forest,0.654545
125,Time-Frequency Domain,"Delta, Theta, Alpha, Beta, Gamma",FP1,"[delta_total_power, delta_avg_power, delta_rel...",StratifiedKFold,SelectKBest - mutual_info_classif,"[delta_total_power, delta_avg_power, theta_rel...",SVM,0.647273
53,Time-Frequency Domain,1-50 Hz,FP1,"['age', 'gender', 'total_power', 'avg_power', ...",StratifiedKFold,SelectKBest - mutual_info_classif,"['age', 'total_power', 'avg_power', 'spectral_...",LightGBM,0.647273
54,Time-Frequency Domain,1-50 Hz,FP1,"['age', 'gender', 'total_power', 'avg_power', ...",StratifiedKFold,SelectPercentile - f_classif,"['age', 'gender']",LightGBM,0.647273
55,Time-Frequency Domain,1-50 Hz,FP1,"['age', 'gender', 'total_power', 'avg_power', ...",StratifiedKFold,SelectPercentile - mutual_info_classif,['spectral_entropy'],LightGBM,0.647273
64,Time-Frequency Domain,1-50 Hz,All,"['age', 'gender', 'total_power', 'avg_power', ...",StratifiedKFold,SelectKBest - f_classif,"['age', 'gender', 'total_power', 'avg_power', ...",SVM,0.647273
77,Time-Frequency Domain,1-50 Hz,All,"['age', 'gender', 'total_power', 'avg_power', ...",StratifiedKFold,SelectKBest - mutual_info_classif,"['gender', 'total_power', 'avg_power', 'relati...",Random Forest,0.647273
